# OpenMC models
This notebook contains the code to create and run the OpenMC model of 
a PWR pincell, based on the [example online](https://github.com/openmc-dev/openmc/wiki/Example-Jupyter-Notebooks).
Running the model includes developing one-group cross section data and depletion using 
only UOX or MOX fuel.

In [45]:
import numpy as np
import openmc
import openmc.deplete as od

In [46]:
uox_model = openmc.Model()
mox_model = openmc.Model()
openmc.config['cross_sections'] = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

In [50]:
# Materials
uo2 = openmc.Material(1, "uo2")
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)
uo2.volume = 0.602

zirconium = openmc.Material(name="zirconium")
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

puo2 = openmc.Material(4)
puo2.add_nuclide('Pu239', 0.94)
puo2.add_nuclide('Pu240', 0.06)
puo2.add_nuclide('O16', 2.0)
puo2.set_density('g/cm3', 11.5)

# Create the mixture
mox = openmc.Material.mix_materials([uo2, puo2], [0.97, 0.03], 'wo')
mox.volume = 0.602

uox_model.Materials = openmc.Materials([uo2, zirconium, water])
mox_model.Materials = openmc.Materials([mox, zirconium, water])

uox_model.Materials.export_to_xml()

uox_model.materials.cross_sections = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"
mox_model.materials.cross_sections = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

In [51]:
# Geometry
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_inner_radius = openmc.ZCylinder(r=0.40)
clad_outer_radius = openmc.ZCylinder(r=0.46)

fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

uox_fuel = openmc.Cell(name='fuel')
uox_fuel.fill = uo2
uox_fuel.region = fuel_region

mox_fuel = openmc.Cell(name='fuel')
mox_fuel.fill = mox
mox_fuel.region = fuel_region

gap = openmc.Cell(name='air gap')
gap.region = gap_region

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

pitch = 1.26
box = openmc.rectangular_prism(width=pitch, height=pitch,
                               boundary_type='reflective')
water_region = box & +clad_outer_radius
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

uox_universe = openmc.Universe(cells=(uox_fuel, gap, clad, moderator))
mox_universe = openmc.Universe(cells=(mox_fuel, gap, clad, moderator))

uox_model.geometry = openmc.Geometry(uox_universe)
uox_model.geometry.export_to_xml()
mox_model.geometry = openmc.Geometry(mox_universe)

In [52]:
# Create a point source
point = openmc.stats.Point((0, 0, 0))
source = openmc.Source(space=point)

In [53]:
# Settings
settings = openmc.Settings()
settings.source = source
settings.batches = 100
settings.inactive = 10
settings.particles = 1000
settings.output = {'tallies':True}
uox_model.settings = settings
mox_model.settings = settings

In [54]:
# Generate one-group cross section data
uox_cross_sections = od.MicroXS.from_model(uox_model, 
                                       reaction_domain = uo2, 
                                       chain_file="chain_endfb71_pwr.xml")
mox_cross_sections = od.MicroXS.from_model(mox_model, 
                                       reaction_domain = mox, 
                                       chain_file="chain_endfb71_pwr.xml")

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [55]:
# save cross sections from UOX model for use in Deplete Reactor
uox_cross_sections.to_csv("micro_xs.csv")

In [56]:
uox_ind_op = od.IndependentOperator(uox_model.Materials, 
                                uox_cross_sections, 
                                "chain_endfb71_pwr.xml")
mox_ind_op = od.IndependentOperator(mox_model.Materials, 
                                mox_cross_sections, 
                                "chain_endfb71_pwr.xml")

In [57]:
uox_integrator = od.PredictorIntegrator(uox_ind_op,
                                    np.ones(12*3)*30, # 3 cycles of operation
                                   power = 278.7,
                                   timestep_units='d')
uox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=33696000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=36288000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=38880000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t

In [58]:
! mv depletion_results.h5 uox_depletion_results.h5

In [59]:
mox_integrator = od.PredictorIntegrator(mox_ind_op,
                                    np.ones(12*3)*30,
                                   power = 278.7,
                                   timestep_units='d')
mox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=33696000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=36288000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=38880000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t

In [60]:
! mv depletion_results.h5 mox_depletion_results.h5

In [61]:
# Print depletion material compositions
uox_results = od.Results("uox_depletion_results.h5")
mox_results = od.Results("mox_depletion_results.h5")

In [69]:
spent_uox = uox_results[-1].get_material("1")
spent_mox = mox_results[-1].get_material("9")

In [71]:
#printing in format for Cyclus recipe
total = 0
weight_frac = 0
for nuclide in spent_uox.nuclides:
    total += nuclide.percent
    if nuclide.percent <= 1e-15:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent/100}</comp>  </nuclide>")
    weight_frac += nuclide.percent*A
    total += nuclide.percent

<nuclide>  <id>10010000</id> <comp>1.7600655543063115e-09</comp>  </nuclide>
<nuclide>  <id>20040000</id> <comp>7.165133835421212e-07</comp>  </nuclide>
<nuclide>  <id>60120000</id> <comp>2.11207814143089e-14</comp>  </nuclide>
<nuclide>  <id>60130000</id> <comp>7.146726361490291e-07</comp>  </nuclide>
<nuclide>  <id>70150000</id> <comp>4.0053820795199015e-13</comp>  </nuclide>
<nuclide>  <id>70160000</id> <comp>4.369114962706105e-16</comp>  </nuclide>
<nuclide>  <id>80160000</id> <comp>0.0004454409551484983</comp>  </nuclide>
<nuclide>  <id>80170000</id> <comp>1.0817760482622219e-08</comp>  </nuclide>
<nuclide>  <id>280660000</id> <comp>1.485184480612375e-15</comp>  </nuclide>
<nuclide>  <id>290650000</id> <comp>1.6707473742164805e-15</comp>  </nuclide>
<nuclide>  <id>290670000</id> <comp>9.798058413340417e-15</comp>  </nuclide>
<nuclide>  <id>290690000</id> <comp>1.255627096601321e-17</comp>  </nuclide>
<nuclide>  <id>300660000</id> <comp>2.794993811836276e-13</comp>  </nuclide>
<nuc

In [72]:
pu_weight_frac = (0.00738*239)/weight_frac
pu_weight_frac

0.2926743890386025

In [73]:
total

0.13661090781209703

In [74]:
#printing in format for Cyclus recipe
for nuclide in spent_mox.nuclides:
    if nuclide.percent <= 1e-15:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent/100}</comp>  </nuclide>")

<nuclide>  <id>10010000</id> <comp>1.8770776671948985e-10</comp>  </nuclide>
<nuclide>  <id>20040000</id> <comp>9.25971362362798e-08</comp>  </nuclide>
<nuclide>  <id>60120000</id> <comp>2.2524927949491653e-15</comp>  </nuclide>
<nuclide>  <id>60130000</id> <comp>9.173263529542912e-08</comp>  </nuclide>
<nuclide>  <id>70160000</id> <comp>3.380705933986699e-17</comp>  </nuclide>
<nuclide>  <id>80160000</id> <comp>0.000447769281791141</comp>  </nuclide>
<nuclide>  <id>80170000</id> <comp>1.074047009425417e-09</comp>  </nuclide>
<nuclide>  <id>280660000</id> <comp>2.6752242057325917e-16</comp>  </nuclide>
<nuclide>  <id>290650000</id> <comp>2.570072254662584e-16</comp>  </nuclide>
<nuclide>  <id>290670000</id> <comp>1.503473678103865e-15</comp>  </nuclide>
<nuclide>  <id>300660000</id> <comp>6.216129441963165e-14</comp>  </nuclide>
<nuclide>  <id>300670000</id> <comp>2.9235770762626664e-13</comp>  </nuclide>
<nuclide>  <id>300680000</id> <comp>4.773460208998419e-13</comp>  </nuclide>
<nuc